## theano實作gradient desecent

* theano是什麼
    * One of the main features of Theano is its symbolic differentiation feature. That is, given a <font color='red'>symbolic mathematical expression</font>, Theano can automatically differentiate the expression with respect to some variable within the expression 
    * 如y=x就是一個symbolic mathematical expression，只要呼叫grad([x],y)，就會自動幫你計算gradient偏微分結果
    * 遇到太難微分的方程式，呼叫grad就解決了，超方便


* 用theano來練習最基本的gradient desecent

* 驗證跟之前寫的gd_test.py結果有沒有一樣

In [22]:
import theano
import theano.tensor as T
import numpy as np

* 須注意theano的型別, floatX

In [23]:
floatX = theano.config.floatX
print floatX

float64


## 初始化
* theano shared variables (就當作全域變數)
* 如果用np create array 要定義<font color='red'>dtype=floatX</font>，到時候才能跟同型別相乘或相加
* 我們後面會產生100筆 x data，所以在w和b這兩個變數也要100筆
* w和b預設-1，不要取random，這樣比較好跟gd_test.py對照結果

In [24]:
x = T.vector()
w = theano.shared(np.array([-1. for i in range(100)], dtype=floatX), name='w')
b = theano.shared(np.array([-1. for i in range(100)], dtype=floatX), name='b')

## 定義fuction，假想我們已知真實的f是長這樣，用底下這個f來產生y data
* 主要用意是要來驗證gradient寫得對不對

In [25]:
y = w*x**2 + b
f = theano.function([x], y)

## 定義cost/error/loss function

In [26]:
y_hat = T.vector()
cost = T.sum((y-y_hat)**2)

* 作gradient desecent
* 對w, b偏微分
* 回傳dw, db偏微分結果

In [27]:
dw, db = T.grad(cost, [w,b])

## 定義graident function
* learning rate 設 0.00001 (這組是最穩定的)
* inputs資料 x, y_hat
* outputs偏微分結果 dw, db
* updates迭代更新w, b參數
* <font color='red'>T.sum(dw)</font> => 把每一筆資料的dw都sum起來
    * 最原始gd作法是把每一筆資料都看過(算偏微分加總)，這是以後改成SGD或是Mini-Batch基礎
* 為什麼不是用np.sum?
    * 因為dw是theano variable，要用theano.tensor裡面的sum來做
    * In[127]: dw.type
    * Out[127]: TensorType(float64, vector)

In [28]:
lr = 0.00001
gradient = theano.function(inputs=[x, y_hat],
                           outputs=[dw, db],
                           updates=[(w, w-lr*T.sum(dw)), (b, b-lr*T.sum(db))])

## create data
* 如果用np create array 要定義<font color='red'>dtype=floatX</font>，到時候才能跟同型別相乘或相加
* y = w*x**2 + b
* <font color='red'>已知參數w=1, b=1，因次最後gd跑出來的w, b要接近1才正確</font>

In [29]:
x_data = np.array(np.linspace(-5.0, 5.0, 100), dtype=floatX)
y_data = np.array(x_data**2+1, dtype=floatX)

* epoch 1 次
* iterate 10000次
* 一開始驗證gd_test.py可以將iterate設為1次，檢查跑出來的w和b有沒有一樣，慢慢增加3次5次

In [30]:
T = 1
t = 10000

## 跑迴圈求解w, b
* = train model
* 將x_data, y_data丟進去gradient裡面
* print出w, b結果
    * w, b是theano shared variable，要用get_value取值，set_value給值
    * 因為w, b每一筆都一樣，取第一筆print就好

In [31]:
for i in range(0, T):
    for j in range(0, t):
        gradient(x_data, y_data)        
    print "w=%f, b=%f" % (w.get_value()[0], b.get_value()[0])

w=1.000017, b=0.999734


## 計算error
* = test model
* 把cost定義為theano function

In [32]:
cost_f = theano.function([x, y_hat], cost)
print "error=%f" % (cost_f(x_data, y_data))

error=0.000003
